## RAG With Multi Data Source

In [6]:

## Wikipedia Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [8]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)
print(wiki.name)

wikipedia


In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings())
retriever = vectordb.as_retriever
retriever

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x7758882e9370>>

In [32]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search", 
                                        "Search for information about Langsmith. For any questions about LangSmith, you must use this tool!")

In [42]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [43]:
tools = [wiki, arxiv, retriever_tool]

In [34]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/noshin/.pyenv/versions/3.12.3/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about Langsmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x775889fe91c0>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAI

In [54]:
##LLM
from dotenv import load_dotenv
import os
from langchain_community.chat_models import ChatOllama


load_dotenv()

llm = ChatOllama(model="llama2", temperature=0)

In [25]:
## Prompt Template

from langchain import hub

##get the propt from the hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [55]:
## Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [56]:
## To run agents, we need agent executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [57]:
agent_executor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Langsmith is a small village located in the rolling hills of the countryside, surrounded by lush green fields and picturesque farmland. The village has a population of around 500 residents, who are known for their warm hospitality and strong sense of community.

The village is home to several historic buildings, including the Langsmith Manor, which dates back to the 16th century and is now a popular bed and breakfast. The manor is surrounded by beautifully landscaped gardens and offers luxurious accommodations and fine dining.

Langsmith is also home to several artisanal shops and cafes, where visitors can find unique gifts, local crafts, and delicious treats. The village is particularly famous for its handmade chocolates, which are made using traditional recipes and techniques that have been passed down through generations.

In addition to its charming atmosphere and rich history, Langsmith is also known for its stunning natural beauty. The vil

{'input': 'Tell me about Langsmith',
 'output': '\nLangsmith is a small village located in the rolling hills of the countryside, surrounded by lush green fields and picturesque farmland. The village has a population of around 500 residents, who are known for their warm hospitality and strong sense of community.\n\nThe village is home to several historic buildings, including the Langsmith Manor, which dates back to the 16th century and is now a popular bed and breakfast. The manor is surrounded by beautifully landscaped gardens and offers luxurious accommodations and fine dining.\n\nLangsmith is also home to several artisanal shops and cafes, where visitors can find unique gifts, local crafts, and delicious treats. The village is particularly famous for its handmade chocolates, which are made using traditional recipes and techniques that have been passed down through generations.\n\nIn addition to its charming atmosphere and rich history, Langsmith is also known for its stunning natural

In [44]:
agent_executor.invoke({"input":"what is Heat-bath random walks with Markov bases?"})



> Entering new AgentExecutor chain...

Heat-bath random walks with Markov bases is a stochastic process that combines the concepts of heat-bath random walks and Markov chains. Here's an explanation:

1. Heat-bath random walks: In this process, the particle moves randomly in a potential field, where the potential energy depends on the position of the particle. The motion is driven by thermal fluctuations, which are simulated using a heat bath. The heat bath is a mathematical construct that represents the distribution of particles in a system at thermal equilibrium.
2. Markov bases: A Markov chain is a mathematical model that describes a sequence of random events where the probability of transitioning from one state to another depends only on the current state and not on any previous states. In other words, the future state of the system is independent of its past states.
3. Combining heat-bath random walks and Markov bases: By combining these two concepts, we can create a stochastic p

{'input': 'what is Heat-bath random walks with Markov bases?',
 'output': "\nHeat-bath random walks with Markov bases is a stochastic process that combines the concepts of heat-bath random walks and Markov chains. Here's an explanation:\n\n1. Heat-bath random walks: In this process, the particle moves randomly in a potential field, where the potential energy depends on the position of the particle. The motion is driven by thermal fluctuations, which are simulated using a heat bath. The heat bath is a mathematical construct that represents the distribution of particles in a system at thermal equilibrium.\n2. Markov bases: A Markov chain is a mathematical model that describes a sequence of random events where the probability of transitioning from one state to another depends only on the current state and not on any previous states. In other words, the future state of the system is independent of its past states.\n3. Combining heat-bath random walks and Markov bases: By combining these tw